In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('Data/pca_projected_506_nc_from_resnet_train.csv')
#train_df.reset_index(drop=True, inplace=True)
train_df.head(5)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_497,f_498,f_499,f_500,f_501,f_502,f_503,f_504,f_505,is_iceberg
0,-1.078176,0.358596,-0.215716,-0.372415,-1.290822,0.039064,0.134564,0.787661,-0.613845,0.108084,...,1.777018,-0.151202,0.480548,1.827665,0.479858,0.244300,0.729476,-1.061594,-0.839429,0
1,-1.273724,1.087702,-1.793496,-1.264303,0.042096,0.182288,0.078520,0.412620,0.980071,-1.091136,...,0.117945,-0.056131,-1.075378,-2.459079,0.350045,0.778946,-0.002505,-0.605706,-2.108809,0
2,0.481582,-1.725730,0.391214,-1.857765,0.999916,-0.145043,-0.434842,-0.503897,-0.264354,0.272160,...,0.059628,2.373542,-0.670700,-1.097062,-0.900625,0.762092,0.226062,0.464445,-1.965059,1
3,-1.089239,0.445182,0.128932,0.059036,-0.324501,0.164366,0.122997,1.574274,-0.467762,0.552849,...,0.935038,0.164698,-0.982558,-1.232302,-0.370637,-1.022586,0.878860,-0.016119,0.663128,0
4,-1.038766,0.221048,-0.418525,0.133892,-0.974831,-0.889494,0.028311,1.753731,-0.450525,0.435784,...,-0.918367,0.498966,0.385581,0.517203,-0.014826,1.858226,0.682199,0.906725,-0.257054,0


In [3]:
features = train_df.columns.tolist()
features.remove('is_iceberg')

train_X = train_df[features]
train_y = np.array(train_df['is_iceberg']).reshape((train_X.shape[0],))

print(train_X.shape, train_y.shape)

(1604, 506) (1604,)


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import xgboost
import lightgbm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, SparsePCA, MiniBatchSparsePCA, KernelPCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, accuracy_score

kfold = KFold(5, shuffle=True, random_state=0)

In [9]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, n_estimators=100)
pca = PCA(whiten=False, random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__boosting_type': ['dart', 'gbdt']
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'lgb__boosting_type': 'gbdt'} -0.116621045968
   mean_train_score  mean_test_score  std_test_score  \
1         -0.032865        -0.116621        0.028763   
0         -0.073187        -0.123540        0.022169   

                           params  rank_test_score  
1  {'lgb__boosting_type': 'gbdt'}                1  
0  {'lgb__boosting_type': 'dart'}                2  


In [16]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, n_estimators=100, boosting_type='gbdt')
pca = PCA(whiten=False, svd_solver='randomized', random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__n_estimators': [100, 200, 400, 800],
    'lgb__learning_rate': [0.1, 0.05, 0.01, 0.005]
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)
print(cv_df['params'].tolist())

{'lgb__learning_rate': 0.05, 'lgb__n_estimators': 100} -0.11563557816
    mean_train_score  mean_test_score  std_test_score  \
4          -0.053173        -0.115636        0.026175   
11         -0.039165        -0.116266        0.026461   
5          -0.033234        -0.116307        0.027611   
0          -0.032865        -0.116621        0.028763   
10         -0.063028        -0.118136        0.024898   
15         -0.063071        -0.118168        0.024959   
6          -0.021453        -0.120740        0.030703   
1          -0.021147        -0.121834        0.031711   
7          -0.015080        -0.126744        0.033876   
2          -0.014970        -0.128001        0.034876   
3          -0.012849        -0.130976        0.036043   
9          -0.129394        -0.163704        0.018741   
14         -0.129536        -0.163812        0.018685   
8          -0.252382        -0.271382        0.013413   
13         -0.252635        -0.271632        0.013342   
12         -0.3960

In [17]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, 
                                 n_estimators=100, learning_rate=.05, boosting_type='gbdt')

pca = PCA(whiten=False, svd_solver='randomized', random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__num_leaves': [2, 3, 7, 15, 31, 63],
    'lgb__min_split_gain': [.0, .1, .2]
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)
print(cv_df['params'].tolist())

{'lgb__min_split_gain': 0.2, 'lgb__num_leaves': 15} -0.11544658773
    mean_train_score  mean_test_score  std_test_score  \
15         -0.054303        -0.115447        0.026319   
16         -0.054303        -0.115447        0.026319   
17         -0.054303        -0.115447        0.026319   
9          -0.053572        -0.115556        0.026063   
10         -0.053548        -0.115579        0.026026   
11         -0.053548        -0.115579        0.026026   
4          -0.053173        -0.115636        0.026175   
5          -0.053173        -0.115643        0.026180   
3          -0.053508        -0.116238        0.025849   
2          -0.058423        -0.117316        0.024888   
8          -0.058423        -0.117316        0.024888   
14         -0.058471        -0.117317        0.024889   
1          -0.097403        -0.122375        0.024696   
7          -0.097403        -0.122375        0.024696   
13         -0.097403        -0.122375        0.024696   
0          -0.115947 

In [18]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, 
                                 n_estimators=100, learning_rate=.05, boosting_type='gbdt',
                                 num_leaves=15, min_split_gain=.2)

pca = PCA(whiten=False, svd_solver='randomized', random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__subsample': [1., .8, .6, .4, .2],
    'lgb__colsample_bytree': [1., .8, .6, .4, .2]
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)
print(cv_df['params'].tolist())

{'lgb__colsample_bytree': 1.0, 'lgb__subsample': 1.0} -0.11544658773
    mean_train_score  mean_test_score  std_test_score  \
0          -0.054303        -0.115447        0.026319   
5          -0.054258        -0.117008        0.025114   
6          -0.062528        -0.117501        0.024364   
1          -0.061434        -0.117604        0.023926   
2          -0.073312        -0.120069        0.025598   
7          -0.074029        -0.120953        0.024829   
11         -0.066966        -0.121769        0.024284   
8          -0.088773        -0.121864        0.023310   
10         -0.058836        -0.122552        0.024147   
3          -0.089145        -0.122616        0.025979   
12         -0.078328        -0.124660        0.024033   
13         -0.093593        -0.127141        0.025174   
15         -0.066344        -0.127493        0.023990   
16         -0.074648        -0.127825        0.024259   
17         -0.085187        -0.128655        0.022738   
9          -0.11466

In [19]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, 
                                 n_estimators=100, learning_rate=.05, boosting_type='gbdt',
                                 num_leaves=15, min_split_gain=.2)

pca = PCA(whiten=False, svd_solver='randomized', random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__subsample': [1., .95, .9, .85],
    'lgb__colsample_bytree': [1., .95, .9, .85]
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)
print(cv_df['params'].tolist())

{'lgb__colsample_bytree': 1.0, 'lgb__subsample': 1.0} -0.11544658773
    mean_train_score  mean_test_score  std_test_score  \
0          -0.054303        -0.115447        0.026319   
4          -0.054162        -0.115714        0.026245   
8          -0.054187        -0.115857        0.026426   
5          -0.055299        -0.116719        0.026336   
13         -0.055699        -0.116989        0.026439   
12         -0.054228        -0.117189        0.026329   
2          -0.057175        -0.117212        0.024620   
1          -0.055470        -0.117239        0.025338   
9          -0.055213        -0.117280        0.025298   
6          -0.057453        -0.117743        0.025741   
10         -0.057336        -0.118196        0.025404   
14         -0.057688        -0.118217        0.026630   
11         -0.060119        -0.118332        0.025538   
7          -0.059177        -0.118357        0.026551   
15         -0.060143        -0.118945        0.025779   
3          -0.05933

In [20]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, 
                                 n_estimators=100, learning_rate=.05, boosting_type='gbdt',
                                 num_leaves=15, min_split_gain=.2)

pca = PCA(whiten=False, svd_solver='randomized', random_state=0, n_components=30)
pipe = Pipeline([
    ('pca', pca),
    ('lgb', lg_clf)
])

params = {
    'lgb__reg_alpha': [1., .8, .6, .4, .2, .1, .0],
    'lgb__reg_lambda': [1., .8, .6, .4, .2, .1, .0]
}

gs = GridSearchCV(pipe, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)
print(cv_df['params'].tolist())

{'lgb__reg_alpha': 0.0, 'lgb__reg_lambda': 0.1} -0.11540581195
    mean_train_score  mean_test_score  std_test_score  \
47         -0.055158        -0.115406        0.025432   
48         -0.054303        -0.115447        0.026319   
46         -0.056152        -0.115774        0.025837   
40         -0.056437        -0.115836        0.025787   
41         -0.055563        -0.116099        0.025585   
43         -0.059780        -0.116136        0.025169   
44         -0.058757        -0.116141        0.025362   
45         -0.057290        -0.116213        0.025499   
42         -0.060885        -0.116227        0.025813   
36         -0.060509        -0.116514        0.025184   
30         -0.060330        -0.116608        0.025349   
32         -0.058244        -0.116659        0.025546   
39         -0.057365        -0.116662        0.024795   
27         -0.058947        -0.116727        0.025503   
34         -0.056902        -0.116836        0.025303   
33         -0.057348     

In [4]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, 
                                 n_estimators=100, learning_rate=.05, boosting_type='gbdt',
                                 num_leaves=15, min_split_gain=.2, reg_lambda=.1)


train_df = pd.read_json('Data/pca_projected_30_from_fine_tune_resnet_train.json')
print(train_df.head(5))

features = train_df.columns.tolist()
features.remove('is_iceberg')

train_X = train_df[features]
train_y = np.array(train_df['is_iceberg']).reshape((train_X.shape[0],))

print(train_X.shape, train_y.shape)

print(cross_val_score(lg_clf, X=train_X, y=train_y, scoring='neg_log_loss', cv=kfold))

lg_clf.fit(train_X, train_y)


test_df = pd.read_json('Data/test.json')
test_ids = test_df['id']
del test_df

test_df = pd.read_json('Data/pca_projected_30_from_fine_tune_resnet_test.json')
test_df.sort_index(inplace=True)
test_df.head(5)
test_X = test_df[features]

predictions = lg_clf.predict_proba(test_X)[:,1]
print(predictions)

submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_lgb_pca_30.csv', float_format="%.15f", index=False)

           f_0       f_1      f_10      f_11      f_12      f_13      f_14  \
0    -1.091220  0.450684  0.579754  0.631756  0.175730  0.246387 -0.798950   
1    -1.296368  1.124568 -0.022570  0.925351 -0.571950  1.046042  1.303893   
10    0.183570 -1.341081 -0.838703  2.923561  1.424435 -1.180463  0.400592   
100  -1.218687  0.928520 -0.376063  0.209557 -0.349536 -1.756338  0.076930   
1000 -0.309406 -1.530215  0.572226 -0.036990  0.142234  0.326848  0.178937   

          f_15      f_16      f_17     ...          f_28      f_29       f_3  \
0    -0.052748 -0.472216 -0.184118     ...     -1.122193  1.526892  0.655805   
1     0.040534 -1.066129  0.468961     ...     -0.978814  0.467171  1.264808   
10   -0.320883 -0.340515 -0.131296     ...      0.923873 -0.961926  0.251447   
100   0.668295 -1.337874 -1.051033     ...      0.008738  1.878277  0.109763   
1000  1.578307 -0.153178 -1.820677     ...      0.475919 -2.289491 -1.302198   

           f_4       f_5       f_6       f_7      

# Old training code

In [5]:
print(xgboost.XGBClassifier(n_jobs=4).get_params())

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': None, 'n_estimators': 100, 'n_jobs': 4, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


In [7]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, seed=0, objective='binary:logistic', eval_metric='logloss')
params = {
    'booster': ['dart', 'gbtree']
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'booster': 'dart'} -0.159207970604
   mean_train_score  mean_test_score  std_test_score                 params  \
0         -0.015699        -0.159208        0.031826    {'booster': 'dart'}   
1         -0.015699        -0.159208        0.031826  {'booster': 'gbtree'}   

   rank_test_score  
0                1  
1                1  


In [8]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart')
params = {
    'n_estimators': [200, 400, 800, 1200, 1600],
    'learning_rate': [0.1, 0.05, 0.01, 0.005]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'learning_rate': 0.005, 'n_estimators': 1200} -0.147295292216
    mean_train_score  mean_test_score  std_test_score  \
18         -0.042416        -0.147295        0.025783   
17         -0.069220        -0.147883        0.023979   
11         -0.069102        -0.147886        0.023795   
19         -0.025510        -0.150452        0.027202   
12         -0.025392        -0.150715        0.026987   
5          -0.015804        -0.158000        0.027958   
13         -0.010925        -0.162649        0.028223   
14         -0.006183        -0.173478        0.029533   
6          -0.004321        -0.181741        0.031102   
0          -0.004270        -0.183529        0.034884   
10         -0.139377        -0.186034        0.015738   
16         -0.139543        -0.186079        0.015785   
7          -0.002308        -0.198716        0.032094   
1          -0.002298        -0.200958        0.035170   
8          -0.001855        -0.205807        0.032889   
9          -0.001594     

In [12]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005)
                
params = {
    'max_depth': [1, 3, 5, 7, 9]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'max_depth': 3} -0.147882765844
   mean_train_score  mean_test_score  std_test_score            params  \
1         -0.069220        -0.147883        0.023979  {'max_depth': 3}   
2         -0.028266        -0.151612        0.023184  {'max_depth': 5}   
0         -0.138066        -0.152458        0.020724  {'max_depth': 1}   
3         -0.026659        -0.152860        0.026972  {'max_depth': 7}   
4         -0.026567        -0.153167        0.027042  {'max_depth': 9}   

   rank_test_score  
1                1  
2                2  
0                3  
3                4  
4                5  


In [13]:
xg_clf = xgboost.XGBClassifier(n_jobs=4, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005)
params = {
    'min_child_weight': [1, 2, 4, 6, 8, 10]
    'gamma': [0, 0.1, 0.2]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

KeyboardInterrupt: 

In [14]:
xg_clf = xgboost.XGBClassifier(n_jobs=6, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8)
params = {
    'colsample_bytree': [1., .8, .6, .4, .2],
    'subsample': [1., .8, .6, .4, .2]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'colsample_bytree': 1.0, 'subsample': 0.8} -0.143585299155
    mean_train_score  mean_test_score  std_test_score  \
1          -0.092684        -0.143585        0.022723   
0          -0.088437        -0.143613        0.023551   
2          -0.102406        -0.145123        0.022285   
6          -0.099024        -0.146476        0.022311   
7          -0.107390        -0.147568        0.022134   
5          -0.095441        -0.148552        0.023668   
3          -0.118837        -0.149041        0.021774   
8          -0.121595        -0.150417        0.021840   
11         -0.112972        -0.156757        0.020458   
12         -0.120831        -0.157825        0.020431   
10         -0.108784        -0.159493        0.021847   
13         -0.132032        -0.161072        0.019744   
4          -0.147755        -0.161485        0.020825   
9          -0.147705        -0.162819        0.021399   
14         -0.152582        -0.169873        0.019269   
15         -0.134553        

In [15]:
xg_clf = xgboost.XGBClassifier(n_jobs=6, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8)
params = {
    'colsample_bytree': [1., .95, .9, .85],
    'subsample': [.9, .85, .8, .75, .7]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'colsample_bytree': 1.0, 'subsample': 0.85} -0.143356138068
    mean_train_score  mean_test_score  std_test_score  \
1          -0.090831        -0.143356        0.022237   
7          -0.094013        -0.143523        0.022745   
0          -0.089346        -0.143580        0.022947   
2          -0.092684        -0.143585        0.022723   
5          -0.090772        -0.143614        0.022808   
6          -0.092252        -0.143789        0.022444   
3          -0.094603        -0.143980        0.022185   
12         -0.095264        -0.144174        0.022559   
8          -0.095914        -0.144186        0.022408   
4          -0.096892        -0.144240        0.021775   
10         -0.092174        -0.144455        0.022903   
11         -0.093606        -0.144538        0.022342   
9          -0.098195        -0.144760        0.022157   
16         -0.095040        -0.144802        0.022252   
17         -0.096703        -0.144856        0.022192   
15         -0.093457       

In [17]:
xg_clf = xgboost.XGBClassifier(n_jobs=2, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8,
                               colsample_bytree=1.,
                               subsample=.85)
params = {
    'reg_alpha': [0., .2, .4, .6, .8, 1.],
    'reg_lambda': [0., .2, .4, .6, .8, 1.]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'reg_alpha': 0.0, 'reg_lambda': 0.0} -0.142308869044
    mean_train_score  mean_test_score  std_test_score  \
0          -0.086888        -0.142309        0.022346   
6          -0.087782        -0.142661        0.022245   
1          -0.087603        -0.142719        0.022267   
2          -0.088394        -0.142814        0.022232   
7          -0.088575        -0.142873        0.022497   
12         -0.088867        -0.142968        0.022504   
3          -0.089091        -0.143043        0.022105   
8          -0.089389        -0.143225        0.022317   
18         -0.090191        -0.143242        0.022297   
9          -0.090354        -0.143281        0.022178   
4          -0.089914        -0.143287        0.022169   
13         -0.089855        -0.143296        0.022280   
5          -0.090831        -0.143356        0.022237   
14         -0.090739        -0.143372        0.022169   
15         -0.091555        -0.143413        0.022252   
19         -0.091118        -0.143

In [18]:
xg_clf = xgboost.XGBClassifier(n_jobs=6, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8,
                               colsample_bytree=1.,
                               subsample=.85)
params = {
    'reg_alpha': [0., .05],
    'reg_lambda': [0., .05]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'reg_alpha': 0.0, 'reg_lambda': 0.0} -0.142308869044
   mean_train_score  mean_test_score  std_test_score  \
0         -0.086888        -0.142309        0.022346   
1         -0.087074        -0.142350        0.022236   
2         -0.087116        -0.142395        0.022226   
3         -0.087262        -0.142418        0.022251   

                                    params  rank_test_score  
0    {'reg_alpha': 0.0, 'reg_lambda': 0.0}                1  
1   {'reg_alpha': 0.0, 'reg_lambda': 0.05}                2  
2   {'reg_alpha': 0.05, 'reg_lambda': 0.0}                3  
3  {'reg_alpha': 0.05, 'reg_lambda': 0.05}                4  


In [19]:
xg_clf = xgboost.XGBClassifier(n_jobs=6, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8,
                               colsample_bytree=1.,
                               subsample=.85,
                               reg_alpha=.0,
                               reg_lambda=.0)
params = {
    'random_state': [0, 1, 2, 3, 4, 5]
}

gs = GridSearchCV(xg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'random_state': 0} -0.142308869044
   mean_train_score  mean_test_score  std_test_score               params  \
0         -0.086888        -0.142309        0.022346  {'random_state': 0}   
1         -0.086888        -0.142309        0.022346  {'random_state': 1}   
2         -0.086888        -0.142309        0.022346  {'random_state': 2}   
3         -0.086888        -0.142309        0.022346  {'random_state': 3}   
4         -0.086888        -0.142309        0.022346  {'random_state': 4}   
5         -0.086888        -0.142309        0.022346  {'random_state': 5}   

   rank_test_score  
0                1  
1                1  
2                1  
3                1  
4                1  
5                1  


In [5]:
print(lightgbm.LGBMClassifier(n_jobs=4).get_params())

{'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_bin': 255, 'max_depth': -1, 'min_child_samples': 10, 'min_child_weight': 5, 'min_split_gain': 0.0, 'n_estimators': 10, 'n_jobs': 4, 'num_leaves': 31, 'objective': None, 'random_state': 0, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 50000, 'subsample_freq': 1}


In [6]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, n_estimators=100)
params = {
    'boosting_type': ['dart', 'gbdt']
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'boosting_type': 'dart'} -0.147456417265
   mean_train_score  mean_test_score  std_test_score  \
0         -0.063662        -0.147456        0.021195   
1         -0.019692        -0.153182        0.032813   

                      params  rank_test_score  
0  {'boosting_type': 'dart'}                1  
1  {'boosting_type': 'gbdt'}                2  


In [8]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, objective='binary', random_state=0, boosting_type='dart')
params = {
    'n_estimators': [100, 200, 400, 800],
    'learning_rate': [0.1, 0.05, 0.01, 0.005]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'learning_rate': 0.05, 'n_estimators': 400} -0.142953260489
    mean_train_score  mean_test_score  std_test_score  \
6          -0.044408        -0.142953        0.022675   
1          -0.037767        -0.143472        0.023018   
0          -0.063662        -0.147456        0.021195   
7          -0.023997        -0.148801        0.027458   
2          -0.025454        -0.149098        0.025622   
5          -0.084108        -0.156178        0.018160   
3          -0.015780        -0.158232        0.030288   
11         -0.123110        -0.179539        0.014419   
4          -0.147895        -0.198235        0.014052   
15         -0.247622        -0.278105        0.012298   
10         -0.261789        -0.290695        0.011888   
9          -0.369633        -0.388232        0.008326   
14         -0.401139        -0.417383        0.007489   
8          -0.458342        -0.470542        0.005942   
13         -0.492299        -0.502411        0.005045   
12         -0.556524       

In [11]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05)
params = {
    'num_leaves': [2, 3, 7, 15, 31, 63],
    'min_split_gain': [.0, .1, .2]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'min_split_gain': 0.2, 'num_leaves': 7} -0.141689966591
    mean_train_score  mean_test_score  std_test_score  \
14         -0.049747        -0.141690        0.023308   
9          -0.044263        -0.142117        0.022428   
16         -0.044388        -0.142126        0.022325   
17         -0.044388        -0.142126        0.022325   
10         -0.044167        -0.142142        0.022242   
11         -0.044167        -0.142142        0.022242   
15         -0.044359        -0.142258        0.021590   
2          -0.049720        -0.142365        0.022238   
8          -0.049720        -0.142365        0.022238   
3          -0.044196        -0.142749        0.022281   
5          -0.044251        -0.142908        0.022564   
4          -0.044408        -0.142953        0.022675   
1          -0.112001        -0.146778        0.023006   
7          -0.112001        -0.146778        0.023006   
13         -0.112001        -0.146778        0.023006   
0          -0.137774        -0.

In [12]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2)

params = {
    'subsample': [1., .8, .6, .4, .2],
    'colsample_bytree': [1., .8, .6, .4, .2]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'colsample_bytree': 1.0, 'subsample': 0.6} -0.139383437373
    mean_train_score  mean_test_score  std_test_score  \
2          -0.066348        -0.139383        0.021851   
1          -0.054522        -0.140038        0.022573   
7          -0.070323        -0.140761        0.020654   
0          -0.049747        -0.141690        0.023308   
3          -0.085711        -0.142644        0.023617   
6          -0.059113        -0.143717        0.023326   
5          -0.055333        -0.144224        0.021510   
8          -0.088043        -0.144275        0.021337   
11         -0.068141        -0.146578        0.021674   
12         -0.076466        -0.147078        0.020020   
10         -0.064487        -0.148820        0.019845   
13         -0.093967        -0.151249        0.020343   
4          -0.123644        -0.152072        0.019904   
9          -0.124147        -0.152829        0.021594   
14         -0.124926        -0.155484        0.020006   
15         -0.090128        

In [14]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2)

params = {
    'colsample_bytree': [1., .95, .9, .85],
    'subsample': [.7, .65, .6, .55, .5]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'colsample_bytree': 1.0, 'subsample': 0.65} -0.138964156897
    mean_train_score  mean_test_score  std_test_score  \
1          -0.062931        -0.138964        0.023561   
6          -0.064000        -0.139073        0.022713   
2          -0.066348        -0.139383        0.021851   
17         -0.068724        -0.139531        0.021149   
3          -0.069896        -0.139855        0.019431   
14         -0.075968        -0.139948        0.021660   
5          -0.060694        -0.139979        0.022561   
0          -0.059749        -0.140015        0.024475   
19         -0.076832        -0.140145        0.022471   
7          -0.067339        -0.140179        0.022032   
8          -0.070515        -0.140436        0.020971   
11         -0.064433        -0.140483        0.023455   
12         -0.068081        -0.140489        0.022752   
13         -0.071169        -0.140811        0.020955   
10         -0.061339        -0.140825        0.022971   
18         -0.072166       

In [15]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2,
                                 colsample_bytree=1.,
                                 subsample=.65)

params = {
    'reg_alpha': [1., .8, .6, .4, .2, .1, .0],
    'reg_lambda': [1., .8, .6, .4, .2, .1, .0]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'reg_alpha': 0.0, 'reg_lambda': 0.1} -0.1384341404
    mean_train_score  mean_test_score  std_test_score  \
47         -0.063499        -0.138434        0.022611   
48         -0.062931        -0.138964        0.023561   
27         -0.066687        -0.139132        0.021659   
26         -0.067155        -0.139265        0.021526   
40         -0.064549        -0.139355        0.023350   
43         -0.069075        -0.139413        0.022160   
41         -0.063746        -0.139430        0.023172   
25         -0.068065        -0.139480        0.023170   
39         -0.065466        -0.139537        0.022374   
45         -0.065760        -0.139592        0.022681   
38         -0.066794        -0.139731        0.022304   
46         -0.064332        -0.139783        0.023118   
32         -0.066594        -0.139834        0.022533   
34         -0.064841        -0.140025        0.022977   
19         -0.069114        -0.140148        0.022498   
20         -0.068417        -0.14015

In [16]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2,
                                 colsample_bytree=1.,
                                 subsample=.65)

params = {
    'reg_alpha': [.0, .05],
    'reg_lambda': [.15, .1, .05]
}

gs = GridSearchCV(lg_clf, params, scoring='neg_log_loss', cv=kfold)
gs.fit(train_X, train_y)
print(gs.best_params_ , gs.best_score_)
cv_df = pd.DataFrame().from_dict(gs.cv_results_)
cv_df = cv_df[['mean_train_score', 'mean_test_score', 'std_test_score', 'params', 'rank_test_score']]
cv_df.sort_values(by=['rank_test_score', 'std_test_score'], inplace=True)
print(cv_df)

{'reg_alpha': 0.0, 'reg_lambda': 0.1} -0.1384341404
   mean_train_score  mean_test_score  std_test_score  \
1         -0.063499        -0.138434        0.022611   
5         -0.063913        -0.138617        0.022427   
3         -0.064493        -0.138630        0.023377   
2         -0.063577        -0.138934        0.022928   
0         -0.064217        -0.139119        0.021800   
4         -0.064215        -0.139683        0.022824   

                                    params  rank_test_score  
1    {'reg_alpha': 0.0, 'reg_lambda': 0.1}                1  
5  {'reg_alpha': 0.05, 'reg_lambda': 0.05}                2  
3  {'reg_alpha': 0.05, 'reg_lambda': 0.15}                3  
2   {'reg_alpha': 0.0, 'reg_lambda': 0.05}                4  
0   {'reg_alpha': 0.0, 'reg_lambda': 0.15}                5  
4   {'reg_alpha': 0.05, 'reg_lambda': 0.1}                6  


In [17]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2,
                                 colsample_bytree=1.,
                                 subsample=.65,
                                 reg_alpha=0.,
                                 reg_lambda=0.1)

losses = []
test_times = 10
for i in range(test_times):
    test_k_fold = KFold(5, shuffle=True, random_state=i)
    log_loss = cross_val_score(lg_clf, train_X, train_y, scoring='neg_log_loss', cv=test_k_fold)
    avg_log_loss = -1. * sum(log_loss) / float(len(log_loss))
    losses.append(avg_log_loss)
        
print(losses)
losses = np.array(losses)
print("Overall average: {:.5f}, stdev: {:.4f}".format(losses.mean(), losses.std()))

[0.13841972171192868, 0.14204997002447101, 0.13921843454679936, 0.14047178852624889, 0.14067191310696964, 0.14057144487186415, 0.14119326800798834, 0.14023561077455832, 0.1389122643391732, 0.13793159944214811]
Overall average: 0.13997, stdev: 0.0012


In [20]:
'''
xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=500,
                               learning_rate=0.01,
                               reg_alpha=.0,
                               reg_lambda=.1)

lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=200, 
                                 min_child_samples=5,
                                 num_leaves=7, 
                                 reg_alpha=.4, 
                                 reg_lambda=.2)
'''  
xg_clf = xgboost.XGBClassifier(n_jobs=6, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='dart',
                               n_estimators=800,
                               learning_rate=0.005,
                               gamma=0.2,
                               min_child_weight=8,
                               colsample_bytree=1.,
                               subsample=.85,
                               reg_alpha=.0,
                               reg_lambda=.0)
'''                               
xg_clf = xgboost.XGBClassifier(n_jobs=4, seed=0, 
                               objective='binary:logistic', eval_metric='logloss', 
                               booster='gbtree',
                               n_estimators=1600,
                               learning_rate=0.005,
                               gamma=0.,
                               min_child_weight=10,
                               colsample_bytree=.8,
                               subsample=.75,
                               reg_alpha=.0,
                               reg_lambda=.05)
'''                                 

'''
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=3,
                                 min_child_samples=1,
                                 min_split_gain=.0,
                                 colsample_bytree=1.,
                                 subsample=.85,
                                 reg_alpha=.2, 
                                 reg_lambda=.1,
                                 min_child_weights=.0)
'''
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=400,
                                 learning_rate=0.05,
                                 num_leaves=7,
                                 min_split_gain=.2,
                                 colsample_bytree=1.,
                                 subsample=.65,
                                 reg_alpha=0.,
                                 reg_lambda=0.1)

In [21]:
test_df = pd.read_json('Data/test.json')
print(test_df.shape)
test_df.head(5)
test_ids = test_df['id']
test_df = pd.read_csv('Data/pca_projected_506_from_resnet_test.csv')
#test_df.reset_index(drop=True, inplace=True)
print(test_df.columns)
test_df.sort_index(inplace=True)
test_df.head(5)

(8424, 4)
Index(['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9',
       ...
       'f_497', 'f_498', 'f_499', 'f_500', 'f_501', 'f_502', 'f_503', 'f_504',
       'f_505', 'is_iceberg'],
      dtype='object', length=507)


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_497,f_498,f_499,f_500,f_501,f_502,f_503,f_504,f_505,is_iceberg
0,-0.937001,-0.046966,-0.123941,1.657609,0.387005,-0.935819,0.875177,-1.270511,-0.197012,-0.564091,...,-0.884518,0.744608,1.615025,-0.645318,0.176217,-0.806068,1.856665,-0.350232,-0.865953,0.0
1,0.118640,-1.240150,0.447019,-1.328023,0.787310,-1.598214,-1.202442,1.852692,-0.389426,-1.097326,...,6.759618,-5.750248,1.102224,1.957992,6.739988,-6.828598,-0.316139,0.019994,-0.214197,0.0
2,-1.764784,3.286722,1.683806,-4.562440,1.846337,4.057018,-1.713150,-2.781449,2.058828,-3.035690,...,-12.019593,13.030576,-24.291735,-20.003954,29.469348,-3.469105,-16.500395,-20.140434,15.906663,0.0
3,1.349804,1.111627,-0.250717,0.631179,0.484991,-0.054787,1.577700,-0.875416,0.119568,0.265341,...,-1.307981,0.003071,1.342591,-0.660067,3.665410,0.553088,0.111166,2.663308,-0.349038,0.0
4,-0.776327,-0.104925,-1.166807,-0.152499,0.485535,-1.454540,-0.705199,3.206676,-0.383359,-0.126195,...,-3.667099,-0.170659,6.333236,-5.072867,-3.982328,-2.256037,2.386261,-4.194104,-1.747197,0.0


In [22]:
features = test_df.columns.tolist()
features.remove('is_iceberg')

test_X = test_df[features]

In [23]:
xg_clf.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1.0, eval_metric='logloss', gamma=0.2,
       learning_rate=0.005, max_delta_step=0, max_depth=3,
       min_child_weight=8, missing=None, n_estimators=800, n_jobs=6,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=1, seed=0,
       silent=True, subsample=0.85)

In [24]:
predictions = xg_clf.predict_proba(test_X)[:,1]
print(predictions)

[ 0.01472192  0.66985995  0.01974309 ...,  0.01474441  0.98494661
  0.01969161]


In [25]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_xgboost_1130_pca_506_on_old.csv', float_format="%.15f", index=False)

In [21]:
lg_clf.fit(train_X, train_y)

LGBMClassifier(boosting_type='dart', colsample_bytree=1.0, learning_rate=0.05,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.2, n_estimators=400, n_jobs=4,
        num_leaves=7, objective='binary', random_state=0, reg_alpha=0.0,
        reg_lambda=0.1, silent=True, subsample=0.65,
        subsample_for_bin=50000, subsample_freq=1)

In [22]:
predictions = lg_clf.predict_proba(test_X)[:,1]
print(predictions)

[ 0.01289007  0.42760015  0.02315127 ...,  0.01373905  0.98449361
  0.02838355]


In [23]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_lightgbm_1130_pca_506_on_old.csv', float_format="%.15f", index=False)

In [101]:
from sklearn.model_selection import KFold, StratifiedKFold
import copy
from sklearn.metrics import log_loss, accuracy_score

def get_oof_prediction(model, fold_num = 5):
    kf = kfold
    counter  = 1
    
    final_predictions = np.zeros((len(test_ids),))
    avg_loss = .0
    avg_acc = .0
    
    for train_index, valid_index in kf.split(train_X, train_y):

        _train_X = train_X.loc[train_index].reset_index(drop=True)
        _valid_X = train_X.loc[valid_index].reset_index(drop=True)
        
        _train_y = train_y[train_index]
        _valid_y = train_y[valid_index]
        
        intance = copy.deepcopy(model)
        intance.fit(_train_X, _train_y)
        
        valid_answer = intance.predict_proba(_valid_X)[:,1].reshape((_valid_X.shape[0],))
        valid_answer_logit = intance.predict(_valid_X)
        loss = log_loss(_valid_y, valid_answer)
        acc = accuracy_score(_valid_y, valid_answer_logit)
        
        avg_loss += loss
        avg_acc += acc
        print("loss={:.4f}, acc={:.3f}".format(loss, acc))
        
        predictions = intance.predict_proba(test_X)[:,1]
        predictions = predictions.reshape((predictions.shape[0],))
        final_predictions += predictions
    
    print("Avg. loss:", avg_loss / (fold_num * 1.0))
    print("Avg. acc:", avg_acc / fold_num * 1.0)
    return final_predictions / (fold_num * 1.0)

In [102]:
lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=0,
                                 boosting_type='dart',
                                 n_estimators=200,
                                 learning_rate=0.1,
                                 num_leaves=15,
                                 min_child_samples=20,
                                 colsample_bytree=.9,
                                 subsample=.7,
                                 reg_alpha=0.1,
                                 reg_lambda=0.4)

predictions = get_oof_prediction(lg_clf)
print(predictions)

submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = predictions
submission.to_csv('Submissions/submission_lgb_1126_pca_70_on_old_oof_avg.csv', float_format="%.15f", index=False)

loss=0.1604, acc=0.950
loss=0.1322, acc=0.947
loss=0.1168, acc=0.963
loss=0.1567, acc=0.941
loss=0.1079, acc=0.959
Avg. loss: 0.134816371243
Avg. acc: 0.951999610592
[ 0.01246961  0.45949685  0.02008037 ...,  0.01134609  0.98582233
  0.01585756]


## Output clipping

In [80]:
from sklearn.metrics import log_loss
probs = xg_clf.predict_proba(train_X)[:,1]
logit = xg_clf.predict(train_X)
error_probs = probs[logit != train_y]
#print(error_probs)
max_error_prob = max(error_probs)
min_error_prob = min(error_probs)
print(max_error_prob, min_error_prob)
print(probs)
print("Original log loss:", log_loss(train_y, probs))

clipped_probs = xg_clf.predict_proba(train_X)[:,1].reshape((train_X.shape[0],)).astype(np.float32)
count = 0
for i, prob in enumerate(clipped_probs):
    if prob > .97:
        clipped_probs[i] = 0.9999999
        count += 1
    elif prob < .03:
        clipped_probs[i] = 0.0000001
        count += 1
    else:
        pass

clipped_probs = np.array(clipped_probs).reshape((clipped_probs.shape[0],))
print('clipped #=', count, ' total#=', clipped_probs.shape[0])
print(clipped_probs)    
print("Clipped log loss:", log_loss(train_y, clipped_probs))

0.965868 0.0395036
[ 0.00979236  0.01160072  0.94080621 ...,  0.06268214  0.01758832
  0.12943955]
Original log loss: 0.0745857695122
clipped #= 1273  total#= 1604
[  1.00000001e-07   1.00000001e-07   9.40806210e-01 ...,   6.26821443e-02
   1.00000001e-07   1.29439548e-01]
Clipped log loss: 0.0639064493717


In [81]:
clipped_probs = xg_clf.predict_proba(test_X)[:,1]
count = 0
print(clipped_probs)
for i, prob in enumerate(clipped_probs):
    if prob > .97:
        clipped_probs[i] = 0.9999999
        count += 1
    elif prob < .03:
        clipped_probs[i] = 0.0000001
        count += 1
    else:
        pass

clipped_probs = np.array(clipped_probs).reshape((clipped_probs.shape[0],))
print('clipped #=', count, ' total#=', clipped_probs.shape[0])
print(clipped_probs)
print(clipped_probs[0])

[ 0.00921421  0.86483711  0.00995382 ...,  0.00742945  0.99213684
  0.0262079 ]
clipped #= 6182  total#= 8424
[  1.00000001e-07   8.64837110e-01   1.00000001e-07 ...,   1.00000001e-07
   9.99999881e-01   1.00000001e-07]
1e-07


In [82]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = clipped_probs
submission.to_csv('Submissions/submission_xgboost_1126_fine_tune_clipped.csv', float_format="%.15f", index=False)

## Several seeds blending

In [25]:
repeat_count = 50

In [49]:
xgb_predictions = np.zeros((test_X.shape[0],))
for i in range(repeat_count):
    xg_clf = xgboost.XGBClassifier(n_jobs=4, 
                               booster='gbtree', 
                               objective='binary:logistic',
                               max_depth=3, 
                               min_child_weight=4, 
                               gamma=.1,
                               n_estimators=510, # slightly more
                               learning_rate=0.01,
                               reg_alpha=.01, # slight more regularization
                               reg_lambda=.12, # slight more regularization
                               random_state=i)
    xg_clf.fit(train_X, train_y)
    xgb_predictions += xg_clf.predict_proba(test_X)[:,1]
    
xgb_predictions /= (repeat_count*1.0)
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = xgb_predictions
submission.to_csv('Submissions/submission_xgboost_blend{}.csv'.format(repeat_count), float_format="%.15f", index=False)

In [27]:
lg_predictions = np.zeros((test_X.shape[0],))
for i in range(repeat_count):
    lg_clf = lightgbm.LGBMClassifier(n_jobs=4, 
                                 objective='binary', 
                                 random_state=i,
                                 boosting_type='dart',
                                 n_estimators=200, 
                                 min_child_samples=1,
                                 num_leaves=7,
                                 min_split_gain=.1,
                                 colsample_bytree=.95,
                                 subsample=.9,
                                 reg_alpha=.15, 
                                 reg_lambda=.15)
    lg_clf.fit(train_X, train_y)
    lg_predictions += lg_clf.predict_proba(test_X)[:,1]
    
lg_predictions /= (repeat_count*1.0)
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = lg_predictions
submission.to_csv('Submissions/submission_lightgbm_blend{}_1119_fine_tune.csv'.format(repeat_count), float_format="%.15f", index=False)